### Importing Libraries 

In [111]:
!pip install geopy
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
import requests
import urllib.request
import time
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.request import urlopen
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## webscraping wikipedia table

In [94]:
# Load url, turn into soup and get tables.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
My_table = soup.find('table',{'class':'wikitable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea


###  Obtain the data of the wikipedia table

In [95]:
Postal_Code=[]
Borough=[]
Neighborhood=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postal_Code.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True).strip('\n').replace('/',','))
print(Neighborhood)

['', '', 'Parkwoods', 'Victoria Village', 'Regent Park , Harbourfront', 'Lawrence Manor , Lawrence Heights', "Queen's Park , Ontario Provincial Government", '', 'Islington Avenue', 'Malvern , Rouge', '', 'Don Mills', 'Parkview Hill , Woodbine Gardens', 'Garden District, Ryerson', 'Glencairn', '', '', 'West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale', 'Rouge Hill , Port Union , Highland Creek', '', 'Don Mills', 'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale', '', '', 'Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood', 'Guildwood , Morningside , West Hill', '', '', 'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks', '', '', '', 'Woburn', '', '', 'Leaside', 'Central Bay Street', 'Christie', '', '', '', 'Cedarbrae', 'Hillcrest Village', 'Bathurst Manor , Wilson Heights , Downsview North', 'Thorncliffe Park', 'Richmond , Adelaide , King', 'Dufferin , Dovercourt Village', '', '', '', 'Scarborough Village', 'Fairview , Henry Farm , Oriole

### Creation of Dataframe , droping of no assigned Borough cells , grouping of the same Postal code and processing no assigned neihberhoods

In [96]:
#Create Dataframe
df=pd.DataFrame(Postal_code,columns=['Postal_code'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood
df['Borough'] = df['Borough'].str.strip()
df['Postal_code'] = df['Postal_code'].str.strip()
df['Neighborhood'] = df['Neighborhood'].str.strip()
df.reset_index(drop = True, inplace = True)
df.head()

,Postal_code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [97]:
#Drop Not assighed Borough rows
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)
#Replace Not assigned Neighborhood with corresponding Borough value
df.loc[df['Neighborhood']== 'Not assigned', 'Neighborhood'] = df['Borough']
#Group by Postal code with ',' join in the neighborhood field
df.groupby('Postal_code')['Neighborhood'].apply(lambda Neighborhood: ' , '.join(Neighborhood)).reset_index()
df.reset_index(drop = True, inplace = True)
df.head()

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [98]:
#df shape
print('the shape of the dataframe is :',df.shape)

the shape of the dataframe is : (103, 3)


### Create Dataframe from csv File and merge dataframes

In [124]:
#create dataframe form csv file
data = pd.read_csv("https://cocl.us/Geospatial_data")
data.rename(columns={'Postal Code':'Postal_code'}, inplace=True)
data.head()

 

,Postal_code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [105]:
#merge dataframes
dfinal = df.merge(data, on="Postal_code", how = 'inner')
dfinal.head()


,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [127]:
dfinal.shape

(103, 5)